d-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 1200px">
</div>

# Hyperopt

The [Hyperopt library](https://github.com/hyperopt/hyperopt) allows for parallel hyperparameter tuning using either random search or Tree of Parzen Estimators (TPE). With MLflow, we can record the hyperparameters and corresponding metrics for each hyperparameter combination. You can read more on [SparkTrials w/ Hyperopt](https://github.com/hyperopt/hyperopt/blob/master/docs/templates/scaleout/spark.md).

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) In this lesson you:
 - Use Hyperopt to train and optimize a feed-forward neural net

In [3]:
%run "./Includes/Classroom-Setup"

In [4]:
from sklearn.datasets.california_housing import fetch_california_housing
from sklearn.model_selection import train_test_split

cal_housing = fetch_california_housing()

# split 80/20 train-test
X_train, X_test, y_train, y_test = train_test_split(cal_housing.data,
                                                    cal_housing.target,
                                                    test_size=0.2,
                                                    random_state=1)

## Data Standardization
Let's do feature-wise standardization.

In [6]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

-sandbox

## Keras Model

We will define our NN in Keras and use the hyperparameters given by HyperOpt.

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> We need to import `tensorflow` within the function due to a pickling issue.  <a href="https://docs.databricks.com/applications/deep-learning/single-node-training/tensorflow.html#tensorflow-2-known-issues" target="_blank">See known issues here.</a>

In [8]:
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
import mlflow
import mlflow.keras
tf.random.set_seed(42)

def create_model(hpo):
  model = Sequential()
  model.add(Dense(int(hpo["dense_l1"]), input_dim=8, activation="relu"))
  model.add(Dense(int(hpo["dense_l2"]), activation="relu"))
  model.add(Dense(1, activation="linear"))
  return model

In [9]:
from hyperopt import fmin, hp, tpe, STATUS_OK, SparkTrials

def runNN(hpo):
  # Need to include the TF import due to serialization issues
  import tensorflow as tf
  
  model = create_model(hpo)

  # Select Optimizer
  optimizer_call = getattr(tf.keras.optimizers, hpo["optimizer"])
  optimizer = optimizer_call(learning_rate=hpo["learning_rate"])

  # Compile model
  model.compile(loss="mse",
                optimizer=optimizer,
                metrics=["mse"])

  history = model.fit(X_train, y_train, validation_split=.2, epochs=10, verbose=2)

  # Evaluate our model
  score = model.evaluate(X_test, y_test, verbose=0)
  obj_metric = score[0]  
  return {"loss": obj_metric, "status": STATUS_OK}

### Setup hyperparameter space and training

We need to create a search space for HyperOpt and set up SparkTrials to allow HyperOpt to run in parallel using Spark worker nodes. We can also start a MLflow run to automatically track the results of HyperOpt's tuning trials.

In [11]:
space = {
  "dense_l1": hp.quniform("dense_l1", 10, 30, 1),
  "dense_l2": hp.quniform("dense_l2", 10, 30, 1),
  "learning_rate": hp.loguniform("learning_rate", -5, 0),
  "optimizer": hp.choice("optimizer", ["Adadelta", "Adam"])
 }

spark_trials = SparkTrials(parallelism=4)

with mlflow.start_run():
  best_hyperparam = fmin(fn=runNN, 
                         space=space, 
                         algo=tpe.suggest, 
                         max_evals=30, 
                         trials=spark_trials)

best_hyperparam

Hyperopt with SparkTrials will automatically track trials in MLflow. To view the MLflow experiment associated with the notebook, click the 'Runs' icon in the notebook context bar on the upper right. There, you can view all runs.
To view logs from trials, please check the Spark executor logs. To view executor logs, expand 'Spark Jobs' above until you see the (i) icon next to the stage from the trial job. Click it and find the list of tasks; Task 0 is the first trial attempt, and subsequent Tasks are retries. Click the 'stderr' link for a task to view trial logs.
 0%| | 0/30 [00:00<?, ?trial/s, best loss=?] 3%|▎ | 1/30 [00:12<05:51, 12.11s/trial, best loss: 0.6919485926628113] 10%|█ | 3/30 [00:15<04:01, 8.94s/trial, best loss: 0.3245863914489746] 13%|█▎ | 4/30 [00:16<02:50, 6.56s/trial, best loss: 0.3245863914489746] 17%|█▋ | 5/30 [00:23<02:47, 6.70s/trial, best loss: 0.3245863914489746] 20%|██ | 6/30 [00:26<02:14, 5.59s/trial, best loss: 0.3245863914489746] 23%|██▎ | 7/30 [00:27<01:36, 4.22s/trial, best loss: 0.3245863914489746] 27%|██▋ | 8/30 [00:28<01:11, 3.25s/trial, best loss: 0.3245863914489746] 30%|███ | 9/30 [00:36<01:38, 4.68s/trial, best loss: 0.3245863914489746] 37%|███▋ | 11/30 [00:38<01:07, 3.58s/trial, best loss: 0.3245863914489746] 40%|████ | 12/30 [00:40<00:56, 3.11s/trial, best loss: 0.3245863914489746] 43%|████▎ | 13/30 [00:48<01:17, 4.58s/trial, best loss: 0.3245863914489746] 47%|████▋ | 14/30 [00:49<00:56, 3.51s/trial, best loss: 0.3245863914489746] 53%|█████▎ | 16/30 [00:51<00:38, 2.76s/trial, best loss: 0.3245863914489746] 57%|█████▋ | 17/30 [00:59<00:56, 4.34s/trial, best loss: 0.3245863914489746] 60%|██████ | 18/30 [01:01<00:43, 3.64s/trial, best loss: 0.3245863914489746] 63%|██████▎ | 19/30 [01:02<00:31, 2.85s/trial, best loss: 0.3245863914489746] 67%|██████▋ | 20/30 [01:04<00:26, 2.60s/trial, best loss: 0.3245863914489746] 70%|███████ | 21/30 [01:10<00:32, 3.63s/trial, best loss: 0.3245863914489746] 77%|███████▋ | 23/30 [01:13<00:20, 2.99s/trial, best loss: 0.3245863914489746] 80%|████████ | 24/30 [01:16<00:18, 3.03s/trial, best loss: 0.3234902620315552] 83%|████████▎ | 25/30 [01:22<00:19, 3.92s/trial, best loss: 0.3234902620315552] 87%|████████▋ | 26/30 [01:25<00:14, 3.65s/trial, best loss: 0.3234902620315552] 90%|█████████ | 27/30 [01:26<00:08, 2.86s/trial, best loss: 0.3234902620315552] 93%|█████████▎| 28/30 [01:28<00:05, 2.60s/trial, best loss: 0.3234902620315552] 97%|█████████▋| 29/30 [01:34<00:03, 3.62s/trial, best loss: 0.3234902620315552]100%|██████████| 30/30 [01:35<00:00, 2.86s/trial, best loss: 0.31949880719184875]100%|██████████| 30/30 [01:35<00:00, 3.19s/trial, best loss: 0.31949880719184875]
Total Trials: 30: 30 succeeded, 0 failed, 0 cancelled.
Out[10]: {'dense_l1': 20.0,
 'dense_l2': 27.0,
 'learning_rate': 0.006992510192068646,
 'optimizer': 1}

To view the MLflow experiment associated with the notebook, click the Runs icon in the notebook context bar on the upper right. There, you can view all runs. You can also bring up the full MLflow UI by clicking the button on the upper right that reads View Experiment UI when you hover over it.

To understand the effect of tuning a hyperparameter:

0. Select the resulting runs and click Compare.
0. In the Scatter Plot, select a hyperparameter for the X-axis and loss for the Y-axis.

-sandbox
&copy; 2020 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>